# BombCell quality metrics in `Params` that change classification outcomes

This notebook is focused on the `Params` thresholds used during BombCell unit type assignment (NOISE / GOOD / MUA / NON-SOMA).

Source of truth used here: `py_bombcell/bombcell/quality_metrics.py`, function `get_quality_unit_type(...)`.


## Giant overview cell: every adjustable quality metric threshold and direction-of-change effect

| Param key | Quality metric compared | Rule (from code) | If you increase param value | If you decrease param value |
|---|---|---|---|---|
| `maxNPeaks` | `nPeaks` | Noise if `nPeaks > maxNPeaks` | **Less strict** (fewer units marked NOISE for extra peaks) | **More strict** (more units become NOISE) |
| `maxNTroughs` | `nTroughs` | Noise if `nTroughs > maxNTroughs` | Less strict | More strict |
| `minWvDuration` | `waveformDuration_peakTrough` | Noise if `waveformDuration_peakTrough < minWvDuration` | More strict for short waveforms (more NOISE) | Less strict |
| `maxWvDuration` | `waveformDuration_peakTrough` | Noise if `waveformDuration_peakTrough > maxWvDuration` | Less strict for long waveforms | More strict |
| `maxWvBaselineFraction` | `waveformBaselineFlatness` | Noise if `waveformBaselineFlatness > maxWvBaselineFraction` | Less strict | More strict |
| `maxScndPeakToTroughRatio_noise` | `scndPeakToTroughRatio` | Noise if `scndPeakToTroughRatio > maxScndPeakToTroughRatio_noise` | Less strict | More strict |
| `minSpatialDecaySlope` *(linear mode)* | `spatialDecaySlope` | Noise if `spatialDecaySlope < minSpatialDecaySlope` | More strict | Less strict |
| `minSpatialDecaySlopeExp` *(exp mode)* | `spatialDecaySlope` | Noise if `spatialDecaySlope < minSpatialDecaySlopeExp` | More strict (reject low slopes) | Less strict |
| `maxSpatialDecaySlopeExp` *(exp mode)* | `spatialDecaySlope` | Noise if `spatialDecaySlope > maxSpatialDecaySlopeExp` | Less strict (allow higher slopes) | More strict |
| `maxPercSpikesMissing` | `percentageSpikesMissing_gaussian` | MUA if `percentageSpikesMissing_gaussian > maxPercSpikesMissing` | Less strict (fewer MUA) | More strict (more MUA) |
| `minNumSpikes` | `nSpikes` | MUA if `nSpikes < minNumSpikes` | More strict (more low-spike units become MUA) | Less strict |
| `maxRPVviolations` | `fractionRPVs_estimatedTauR` | MUA if `fractionRPVs_estimatedTauR > maxRPVviolations` | Less strict | More strict |
| `minPresenceRatio` | `presenceRatio` | MUA if `presenceRatio < minPresenceRatio` | More strict | Less strict |
| `minAmplitude` *(only when `extractRaw=True`)* | `rawAmplitude` | MUA if `rawAmplitude < minAmplitude` | More strict | Less strict |
| `minSNR` *(only when `extractRaw=True`)* | `signalToNoiseRatio` | MUA if `signalToNoiseRatio < minSNR` | More strict | Less strict |
| `maxDrift` *(only when `computeDrift=True`)* | `maxDriftEstimate` | MUA if `maxDriftEstimate > maxDrift` | Less strict | More strict |
| `isoDmin` *(only when `computeDistanceMetrics=True`)* | `isolationDistance` | MUA if `isolationDistance < isoDmin` | More strict | Less strict |
| `lratioMax` *(only when `computeDistanceMetrics=True`)* | `Lratio` | MUA if `Lratio > lratioMax` | Less strict | More strict |
| `minTroughToPeak2Ratio_nonSomatic` | `troughToPeak2Ratio` | Non-somatic condition uses `troughToPeak2Ratio < minTroughToPeak2Ratio_nonSomatic` | Easier to satisfy non-somatic condition | Harder to satisfy |
| `minWidthFirstPeak_nonSomatic` | `mainPeak_before_width` | Non-somatic condition uses `mainPeak_before_width < minWidthFirstPeak_nonSomatic` | Easier to satisfy non-somatic condition | Harder to satisfy |
| `minWidthMainTrough_nonSomatic` | `mainTrough_width` | Non-somatic condition uses `mainTrough_width < minWidthMainTrough_nonSomatic` | Easier to satisfy non-somatic condition | Harder to satisfy |
| `maxPeak1ToPeak2Ratio_nonSomatic` | `peak1ToPeak2Ratio` | Non-somatic condition uses `peak1ToPeak2Ratio > maxPeak1ToPeak2Ratio_nonSomatic` | Harder to satisfy non-somatic condition | Easier to satisfy |
| `maxMainPeakToTroughRatio_nonSomatic` | `mainPeakToTroughRatio` | Non-somatic condition uses `mainPeakToTroughRatio > maxMainPeakToTroughRatio_nonSomatic` | Harder to satisfy non-somatic condition | Easier to satisfy |

> Notes:
> - Spatial decay thresholds only participate when `computeSpatialDecay=True`.
> - Distance and drift thresholds only participate when their compute flags are enabled.
> - Non-somatic assignment is applied after NOISE/MUA/GOOD logic.


## maxNPeaks

`nPeaks > maxNPeaks` marks a unit as **NOISE**.

- Increase `maxNPeaks`: allows more complex waveforms, fewer units marked noise.
- Decrease `maxNPeaks`: stricter waveform-shape filtering, more units marked noise.

![nPeaks threshold example](./nPeaks_maxNPeaks.png)


## maxNTroughs

`nTroughs > maxNTroughs` marks a unit as **NOISE**.

- Increase: less strict on multi-trough waveforms.
- Decrease: more strict, more units become noise.

![Peaks/troughs example](./docs/images/numberTroughsPeaks.png)


## minWvDuration

`waveformDuration_peakTrough < minWvDuration` marks **NOISE**.

- Increase: rejects more short-duration waveforms.
- Decrease: allows shorter waveforms to stay non-noise.

![Waveform examples](./docs/images/goodWaveforms.png)


## maxWvDuration

`waveformDuration_peakTrough > maxWvDuration` marks **NOISE**.

- Increase: allows broader waveforms.
- Decrease: rejects broad waveforms sooner.

![Waveform examples](./docs/images/badWaveforms.png)


## maxWvBaselineFraction

`waveformBaselineFlatness > maxWvBaselineFraction` marks **NOISE**.

- Increase: tolerates noisier/non-flat baseline regions.
- Decrease: stricter baseline quality requirement.

![Baseline flatness example](./BaselineFlatness.png)


## maxScndPeakToTroughRatio_noise

`scndPeakToTroughRatio > maxScndPeakToTroughRatio_noise` marks **NOISE**.

- Increase: allows larger secondary-peak-to-trough ratio.
- Decrease: stricter waveform morphology filter.

![Second peak to trough ratio](./ScndPeakToTroughRatio.png)


## minSpatialDecaySlope (linear fit mode)

When `computeSpatialDecay=True` and `spDecayLinFit=True`,
`spatialDecaySlope < minSpatialDecaySlope` marks **NOISE**.

- Increase: stricter (more units fail low-slope requirement).
- Decrease: less strict.

![Spatial decay concept](./docs/images/waveformsSingleNoise.png)


## minSpatialDecaySlopeExp

In exponential spatial-decay mode (`spDecayLinFit=False`),
`spatialDecaySlope < minSpatialDecaySlopeExp` marks **NOISE**.

- Increase: stricter lower bound.
- Decrease: less strict lower bound.

![Spatial decay slope exp](./SpatialDecaySlopeExp.png)


## maxSpatialDecaySlopeExp

In exponential spatial-decay mode,
`spatialDecaySlope > maxSpatialDecaySlopeExp` marks **NOISE**.

- Increase: less strict upper bound.
- Decrease: stricter upper bound.

![Spatial decay slope exp](./SpatialDecaySlopeExp.png)


## maxPercSpikesMissing

`percentageSpikesMissing_gaussian > maxPercSpikesMissing` marks **MUA** (if not already noise).

- Increase: fewer units moved to MUA for missing spikes.
- Decrease: more units moved to MUA.

![Missing spikes and drift](./docs/images/percSpikesMissingDrift.png)


## minNumSpikes

`nSpikes < minNumSpikes` marks **MUA**.

- Increase: stricter minimum activity requirement.
- Decrease: allows low-spike units to remain GOOD more often.

![Histogram style QC context](./docs/images/generalPlotHist.png)


## maxRPVviolations

`fractionRPVs_estimatedTauR > maxRPVviolations` marks **MUA**.

- Increase: more tolerant of refractory-period violations.
- Decrease: stricter contamination control.

![RPV threshold](./fractionRPVs_estimated_TauR.png)


## minPresenceRatio

`presenceRatio < minPresenceRatio` marks **MUA**.

- Increase: stricter temporal-stability requirement.
- Decrease: less strict.

![Global QC histogram context](./docs/images/generalPlotHist.png)


## minAmplitude

(Only when `extractRaw=True`) `rawAmplitude < minAmplitude` marks **MUA**.

- Increase: stricter amplitude requirement.
- Decrease: allows lower-amplitude units to remain GOOD more often.

![Amplitude metric](./docs/images/amplitude.png)


## minSNR

(Only when `extractRaw=True`) `signalToNoiseRatio < minSNR` marks **MUA**.

- Increase: stricter SNR requirement.
- Decrease: less strict SNR requirement.

![Raw waveform/SNR context](./docs/images/GUI_rawWv.png)


## maxDrift

(Only when `computeDrift=True`) `maxDriftEstimate > maxDrift` marks **MUA**.

- Increase: allows larger drift before MUA assignment.
- Decrease: stricter drift tolerance.

![Drift context](./docs/images/percSpikesMissingDrift.png)


## isoDmin

(Only when `computeDistanceMetrics=True`) `isolationDistance < isoDmin` marks **MUA**.

- Increase: stricter isolation requirement.
- Decrease: less strict isolation requirement.

![Isolation distance](./docs/images/isolationDistance.png)


## lratioMax

(Only when `computeDistanceMetrics=True`) `Lratio > lratioMax` marks **MUA**.

- Increase: less strict on cluster overlap.
- Decrease: stricter overlap filtering.

![Isolation/L-ratio context](./docs/images/GUI_isoD.png)


## minTroughToPeak2Ratio_nonSomatic

Non-somatic condition includes `troughToPeak2Ratio < minTroughToPeak2Ratio_nonSomatic`.

- Increase: makes this non-somatic criterion easier to satisfy.
- Decrease: makes it harder.

![Peaks/troughs morphology](./docs/images/numberTroughsPeaks.png)


## minWidthFirstPeak_nonSomatic

Non-somatic condition includes `mainPeak_before_width < minWidthFirstPeak_nonSomatic`.

- Increase: easier to satisfy (more potential NON-SOMA).
- Decrease: harder to satisfy.

![Waveform morphology](./docs/images/goodWaveforms.png)


## minWidthMainTrough_nonSomatic

Non-somatic condition includes `mainTrough_width < minWidthMainTrough_nonSomatic`.

- Increase: easier to satisfy non-somatic criterion.
- Decrease: harder to satisfy.

![Waveform morphology](./docs/images/badWaveforms.png)


## maxPeak1ToPeak2Ratio_nonSomatic

Non-somatic condition includes `peak1ToPeak2Ratio > maxPeak1ToPeak2Ratio_nonSomatic`.

- Increase: harder for units to satisfy this criterion.
- Decrease: easier to satisfy this criterion.

![Peaks/troughs morphology](./docs/images/numberTroughsPeaks.png)


## maxMainPeakToTroughRatio_nonSomatic

Non-somatic condition includes `mainPeakToTroughRatio > maxMainPeakToTroughRatio_nonSomatic`.

- Increase: harder to satisfy this criterion.
- Decrease: easier to satisfy this criterion.

![Waveform morphology](./docs/images/numberTroughsPeaks.png)
